In [49]:
%run setup.py
%load_ext autoreload 
%autoreload 2

from keplergl import KeplerGl
import geopandas as gpd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
gdf = gpd.read_file(abspath(join('..', 'data', 'sections_generated.gpkg')))
import cba as cba
#from cba_result import CbaResult

cba_model = cba.CostBenefitAnalysisModel()

input = gdf.apply(lambda row: Section.from_row(row), axis=1)
output = [cba_model.compute_cba_for_section(i) for i in input]

In [68]:
from roads_cba_py.cba_result import CbaResult
from roads_cba_py.section import Section

example_data_dir = abspath(os.path.join(os.getcwd(), '..', 'tests', 'example_data'))


files = [f for f in glob.glob(join(example_data_dir, "section_*.json")) if "output" not in f]
idents = [re.match(".*section_(.*).json", f)[1] for f in files]
#idents = ["637227_299"]
def f(ident):
    rv = CbaResult.from_file(join(example_data_dir, f"section_{ident}.output.json")).to_dict()
    rv['orma_way_id'] = ident
    return rv

inputs = [Section.from_file(join(example_data_dir, f"section_{ident}.json")).__dict__['_data'] for ident in idents[0:5]]
outputs = [f(ident) for ident in idents[0:5]]
input_df = pd.DataFrame(inputs)
output_df = pd.DataFrame(outputs)

In [43]:
gdf = gpd.read_file(abspath(join('..', 'data', 'sections_generated.gpkg')))
print(gdf.shape)
display(gdf.head())
gdf = gdf.merge(input_df, left_on='way_id_district', right_on='orma_way_id')
print(gdf.shape)

gdf = gdf.merge(output_df, left_on='way_id_district', right_on='orma_way_id')
print(gdf.shape)


(1013, 42)


,VPROMM_ID,way_id_district,road number,name,road start location,road end location,province,district,section_commune_gso,management,x_0,y_0,x_1,y_1,length,section_lanes,width,link_class,section_terrain,section_temperature,section_moisture,surface,section_surface,condition,iri,section_pavement_age,section_traffic,section_motorcycle,section_small_car,section_medium_car,section_four_wheel,section_delivery_vehicle,section_light_truck,section_medium_truck,section_heavy_truck,section_articulated_truck,section_small_bus,section_medium_bus,section_large_bus,aadt,section_traffic_growth,geometry
0,033BQ00001,635950_304,ĐH.01,Quang Minh-Vô Điếm-Bằng Hành,QL279 (Km0+660),QL 279 (Km20+800),Hà Giang,Huyện Bắc Quang,None,3,104.841639,22.392546,104.942579,22.437261,12.877356,3,5,6,3,1,4,1,2,3,None,11,3,None,None,None,None,None,None,None,None,None,None,None,None,None,3,"LINESTRING (104.84164 22.39255, 104.84529 22.3..."
1,033BQ00001,635951_304,ĐH.01,Quang Minh-Vô Điếm-Bằng Hành,QL279 (Km0+660),QL 279 (Km20+800),Hà Giang,Huyện Bắc Quang,None,3,104.942579,22.437261,105.072680,22.383061,16.712919,3,3,4,3,1,4,2,2,4,None,13,3,None,None,None,None,None,None,None,None,None,None,None,None,None,3,"LINESTRING (104.94258 22.43726, 104.94887 22.4..."
2,034QB00057,635952_299,ĐX.57,UBND xã đi Cốc Méo - Lao Chải - Na Cạn -Na Quang,None,None,Hà Giang,Huyện Quản Bạ,None,None,104.997324,23.157872,104.986316,23.161160,1.664770,None,4.0,8,3,1,4,None,4,4,None,13,1,None,None,None,None,None,None,None,None,None,None,None,None,None,3,"LINESTRING (104.99732 23.15787, 104.99733 23.1..."
3,032HG00003,635954_303,ĐT.178,Yên Bình - Cốc Pài,"Km23+600, QL.279, (huyện Quang Bình)","thị trấn Cốc Pài, huyện Xín Mần",Hà Giang,Huyện Xín Mần,None,2,104.535704,22.482215,104.463090,22.669164,41.069903,None,6,5,3,1,4,1,2,3,None,None,3,None,None,None,None,None,None,None,None,None,None,None,None,None,3,"LINESTRING (104.53570 22.48221, 104.53570 22.4..."
4,032HG00003,635954_305,ĐT.178,Yên Bình - Cốc Pài,"Km23+600, QL.279, (huyện Quang Bình)","thị trấn Cốc Pài, huyện Xín Mần",Hà Giang,Huyện Quang Bình,None,2,104.587438,22.449030,104.535704,22.482215,12.008070,None,6,5,3,1,4,1,2,3,None,None,3,None,None,None,None,None,None,None,None,None,None,None,None,None,3,"LINESTRING (104.58744 22.44903, 104.58795 22.4..."


(5, 84)
(5, 178)


In [4]:
df = pd.read_csv(join(fldr, 'RoADS-CBA','RoADS_CBA-Results.csv'))
df.head()

,road_id,Implementation year,Work name,accumulated_budget
0,603159_298,6,Functional Overlay (<=50mm),440.808172
1,603159_301,6,Functional Overlay (<=50mm),429.283672
2,603646_298,6,Functional Overlay (<=50mm),318.787163
3,614761_296,6,Functional Overlay (<=50mm),453.954666
4,614791_297,6,Functional Overlay (<=50mm),400.810519


In [18]:
config_file = join('.', 'kepler_config.json')
with open(config_file, 'r') as file:
    config = eval(''.join(file.readlines()))

In [40]:
map_1 = KeplerGl(height=1000, config=config)
map_1.add_data(data=gdf, name="homogeneous_links")


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [77]:
map_1

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'd6o88tlh', 'type':…

In [9]:
with open(config_file, 'w') as file:
    file.write(str(map_1.config))